In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import pandas as pd

In [2]:
# Load environment variables
load_dotenv()

# Instantiate the OpenAI client with your API key
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Retrieve and print all available model IDs
models = client.models.list()
models = pd.DataFrame(models)
models

,0,1,2,3
0,"(id, gpt-4o-audio-preview-2024-12-17)","(created, 1734034239)","(object, model)","(owned_by, system)"
1,"(id, dall-e-3)","(created, 1698785189)","(object, model)","(owned_by, system)"
2,"(id, text-embedding-3-large)","(created, 1705953180)","(object, model)","(owned_by, system)"
3,"(id, dall-e-2)","(created, 1698798177)","(object, model)","(owned_by, system)"
4,"(id, gpt-4o-audio-preview-2024-10-01)","(created, 1727389042)","(object, model)","(owned_by, system)"
...,...,...,...,...
58,"(id, gpt-4o-mini-audio-preview-2024-12-17)","(created, 1734115920)","(object, model)","(owned_by, system)"
59,"(id, gpt-3.5-turbo-0125)","(created, 1706048358)","(object, model)","(owned_by, system)"
60,"(id, o1-mini-2024-09-12)","(created, 1725648979)","(object, model)","(owned_by, system)"
61,"(id, gpt-4-1106-preview)","(created, 1698957206)","(object, model)","(owned_by, system)"


In [3]:
## Chat completion API

## Completion API
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of the moon?"}
  ],
  temperature=0.7,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
response


ChatCompletion(id='chatcmpl-BNXhgpsF96ZlrCCoKcQHxgeQdaOAV', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The moon does not have a capital as it is not a country or a sovereign entity. The moon is Earth's natural satellite and does not have its own government or capital city.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1744950800, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=37, prompt_tokens=25, total_tokens=62, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [4]:
response.choices[0].message.content

"The moon does not have a capital as it is not a country or a sovereign entity. The moon is Earth's natural satellite and does not have its own government or capital city."

In [6]:
import requests
import json

### Interacting with an external API
#### Fitness App
Here, I am interacting with a fitness API - https://rapidapi.com/ltdbilgisam/api/ai-workout-planner-exercise-fitness-nutrition-guide

In [7]:
def get_exercise_data(payload):
    """
    This function gets the exercise data from the API"""
    url = "https://ai-workout-planner-exercise-fitness-nutrition-guide.p.rapidapi.com/customWorkoutPlan"
    querystring = {"noqueue":"1"}
    headers = {
		"x-rapidapi-key": "00a55e62b3mshf5066d494cddc56p1b045ajsn88ea8da74aa1",
		"x-rapidapi-host": "ai-workout-planner-exercise-fitness-nutrition-guide.p.rapidapi.com",
		"Content-Type": "application/json"
	}
    response = requests.post(url, json=payload, headers=headers, params=querystring) #
    return response.json()

In [8]:
payload = {
	"goal": "Improve overall fitness",
	"fitness_level": "Intermediate",
	"preferences": ["Cardio", "Strength", "Yoga"],
	# "health_conditions": [],
	"schedule": {
		"days_per_week": 5,
		"session_duration": 45
	},
	"plan_duration_weeks": 6,
	"custom_goals": ["Increase flexibility", "Improve core strength", "Gain muscle mass"],
	"lang": "en"
	}

functions = [
    {
        "name": "get_exercise_data",
        "description": "Get the exercise data from the API",
        "parameters": {
            "type": "object",
            "properties": {
                "payload": payload
            }
        }
    }
]

completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a fitness coach. You are given a user's fitness goals and you need to create a workout plan for them."},
        {"role": "user", "content": "I want to have a general fitness plan. I am a 30 year old male with vegetarian diet and I want to have a workout plan for 5 days a week."}
    ],
    functions=functions
)

In [9]:
print(completion.choices[0].message.content)

That's great! I can create a customized workout plan for you based on your goals. Let's include a mix of strength training, cardio, and flexibility exercises to ensure a well-rounded fitness routine. I'll provide you with exercises for each day of the week. Let's get started with the plan!

### Workout Plan for a 30-Year-Old Male (5 Days a Week)

**Day 1: Strength Training**
1. Squats - 4 sets of 12 reps
2. Push-ups - 3 sets of 15 reps
3. Bent-over rows - 3 sets of 12 reps
4. Plank - 3 sets of 30 seconds

**Day 2: Cardio**
1. Jogging or Running - 30 minutes
2. Jumping Jacks - 3 sets of 30 seconds
3. Burpees - 3 sets of 10 reps
4. Jump Rope - 3 sets of 1 minute

**Day 3: Rest or Active Recovery (Yoga or Stretching)**

**Day 4: Full Body Workout**
1. Lunges - 3 sets of 12 reps each leg
2. Dumbbell Shoulder Press - 3 sets of 12 reps
3. Bicycle Crunches - 3 sets of 20 reps each side
4. Russian Twists - 3 sets of 15 reps each side

**Day 5: Cardio and Core**
1. Cycling - 30 minutes
2. Mount

### Weather API

In [14]:
def get_current_weather(location):
    """Get the current weather in a given location"""

    url = "https://ai-weather-by-meteosource.p.rapidapi.com/find_places"

    querystring = {"text":location}

    headers = {
      "x-rapidapi-key": "00a55e62b3mshf5066d494cddc56p1b045ajsn88ea8da74aa1",
      "x-rapidapi-host": "ai-weather-by-meteosource.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    print(response.json())
  
    return response.json()

In [15]:
response=get_current_weather('Mumbai')

[{'name': 'Mumbai', 'place_id': 'mumbai', 'adm_area1': 'Maharashtra', 'adm_area2': None, 'country': 'India', 'lat': '19.07283N', 'lon': '72.88261E', 'timezone': 'Asia/Kolkata', 'type': 'settlement'}, {'name': 'Navi Mumbai', 'place_id': 'navi-mumbai', 'adm_area1': 'Maharashtra', 'adm_area2': 'Thane', 'country': 'India', 'lat': '19.03681N', 'lon': '73.01582E', 'timezone': 'Asia/Kolkata', 'type': 'settlement'}, {'name': 'Borivali', 'place_id': 'borivli', 'adm_area1': 'Maharashtra', 'adm_area2': 'Mumbai Suburban', 'country': 'India', 'lat': '19.23496N', 'lon': '72.85976E', 'timezone': 'Asia/Kolkata', 'type': 'settlement'}, {'name': 'Powai', 'place_id': 'powai', 'adm_area1': 'Maharashtra', 'adm_area2': 'Mumbai Suburban', 'country': 'India', 'lat': '19.1164N', 'lon': '72.90471E', 'timezone': 'Asia/Kolkata', 'type': 'settlement'}, {'name': 'Madh', 'place_id': 'madh-1264586', 'adm_area1': 'Maharashtra', 'adm_area2': 'Mumbai Suburban', 'country': 'India', 'lat': '19.15N', 'lon': '72.83333E', 't

In [16]:
response

[{'name': 'Mumbai',
  'place_id': 'mumbai',
  'adm_area1': 'Maharashtra',
  'adm_area2': None,
  'country': 'India',
  'lat': '19.07283N',
  'lon': '72.88261E',
  'timezone': 'Asia/Kolkata',
  'type': 'settlement'},
 {'name': 'Navi Mumbai',
  'place_id': 'navi-mumbai',
  'adm_area1': 'Maharashtra',
  'adm_area2': 'Thane',
  'country': 'India',
  'lat': '19.03681N',
  'lon': '73.01582E',
  'timezone': 'Asia/Kolkata',
  'type': 'settlement'},
 {'name': 'Borivali',
  'place_id': 'borivli',
  'adm_area1': 'Maharashtra',
  'adm_area2': 'Mumbai Suburban',
  'country': 'India',
  'lat': '19.23496N',
  'lon': '72.85976E',
  'timezone': 'Asia/Kolkata',
  'type': 'settlement'},
 {'name': 'Powai',
  'place_id': 'powai',
  'adm_area1': 'Maharashtra',
  'adm_area2': 'Mumbai Suburban',
  'country': 'India',
  'lat': '19.1164N',
  'lon': '72.90471E',
  'timezone': 'Asia/Kolkata',
  'type': 'settlement'},
 {'name': 'Madh',
  'place_id': 'madh-1264586',
  'adm_area1': 'Maharashtra',
  'adm_area2': 'Mum

In [17]:
functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    
                },
                "required": ["location"],
            },
        }
    ]

In [18]:
functions

[{'name': 'get_current_weather',
  'description': 'Get the current weather in a given location',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'The city and state, e.g. San Francisco, CA'}},
   'required': ['location']}}]

In [20]:
user_message="Hi There"
messages=[]
messages.append({"role": "user", "content":user_message})
completion=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=
       messages
    
)

In [21]:
completion.choices[0].message

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

In [22]:
messages

[{'role': 'user', 'content': 'Hi There'}]

In [57]:
user_message="What is the temperature of Bangalore"

messages= [{"role": "user", "content": user_message}]
completion=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
    
)

In [58]:
messages

[{'role': 'user', 'content': 'What is the temperature of Bangalore'}]

In [59]:
completion

ChatCompletion(id='chatcmpl-BNY4YeRntOxSwKifjxMmcPpPiHThF', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Bangalore"}', name='get_current_weather'), tool_calls=None))], created=1744952218, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=69, total_tokens=86, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [60]:
completion.choices[0].message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Bangalore"}', name='get_current_weather'), tool_calls=None)

In [61]:
response=completion.choices[0].message


In [62]:
response

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Bangalore"}', name='get_current_weather'), tool_calls=None)

In [63]:
function_name=response.function_call.name
print(f"Function name: {function_name}")

Function name: get_current_weather


In [64]:
location = eval(response.function_call.arguments)['location']
print(location)


Bangalore


In [66]:
location, function_name

('Bangalore', 'get_current_weather')

In [67]:
# Step 4: send the info on the function call and function response to GPT

messages.append(response)  # extend conversation with assistant's reply
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": location,
    }
)

In [68]:
messages

[{'role': 'user', 'content': 'What is the temperature of Bangalore'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Bangalore"}', name='get_current_weather'), tool_calls=None),
 {'role': 'function', 'name': 'get_current_weather', 'content': 'Bangalore'}]

In [69]:
# extend conversation with function response
second_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)  # get a new response from GPT where it can see the function response

In [72]:
second_response

ChatCompletion(id='chatcmpl-BNY6VrTSiVc0BFotv8tSJhalEber0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current temperature in Bangalore is 27°C.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1744952339, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=12, prompt_tokens=96, total_tokens=108, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [71]:
second_response.choices[0].message.content

'The current temperature in Bangalore is 27°C.'